### Run Pricing Model
This notebook runs MonteCarlo simulation for each sample in our dataset. 

Further research about the relationship among market price, modeling price and future return can be explored based on the work in this notebook.

In [1]:
import os
import re
import copy
import math
import pickle
import random
import numpy as np
import pandas as pd
import import_ipynb
import scipy.stats as st
import statsmodels.api as sm
from scipy.special import comb
from tqdm import tqdm, trange

import seaborn as sns
from matplotlib import pyplot as plt 

plt.style.use('seaborn')
pd.set_option('display.max_columns',None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [2]:
%load_ext autoreload
%autoreload 2

from PricingModel import BSmodel, TreeModel, MonteCarlo, Hermite, partialHermite

importing Jupyter notebook from PricingModel.ipynb


#### 1. Data Loading

In [3]:
data_path = '../../../../export/scratch/for_yifan/research/'
cbond_info = pd.read_csv(os.path.join(data_path, 'cbond_info.csv'), index_col=False, low_memory=False)
cbond_rate = pd.read_csv(os.path.join(data_path, 'sina_rate.csv'), index_col=False, low_memory=False)
cbond_price = pd.read_csv(os.path.join(data_path, 'cbond_price.csv'), index_col=False, low_memory=False)
cbond_data = pickle.load(open(os.path.join(data_path, 'cbond_data.pkl'), 'rb'))
stock_data = pickle.load(open(os.path.join(data_path, 'stock_data.pkl'), 'rb'))

In [4]:
cbond_info.head(1)

,bond_ticker,stock_ticker,name,listed_date,start_date,exit_date,conv_date,conv_price,rdmpt_bgndt,rdmpt_enddt,rdmpt_transit,rpchs_bgndt,rpchs_enddt,rpchs_transit,term_year,interest_freq,payment,put_win,put_price,call_win,call_min,call_price
0,110030.SH,600185.SH,格力地产,20150113,20170103,20191210,20141223,20.9000,20150630,20191224,130.0000,20161225.0000,20191225.0000,70.0000,5.0000,1,106.0000,30,103,30,15,100


In [5]:
cbond_price.head(1)

,bond_ticker,date,open,high,low,close,volume,amount,pre_close,ovnt_ret,intra_ret,return,next_return,is_last,conv_price,rdmpt_bgndt,rdmpt_enddt,stock_ticker,adj_factor,top1500,adj_conv_price,adj_conv_share
0,110030.SH,20170103,113.8000,114.1500,113.8000,114.0400,3650.0000,4163.4180,NaN,NaN,0.0021,NaN,0.0017,False,7.2600,20150630,20191224,600185.SH,0.1652,False,43.9513,2.2752


In [6]:
cbond_rate.head(1)

,bond_ticker,start_date,end_date,rate
0,110030.SH,20141225,20151224,0.6000


In [7]:
cbond_rate[cbond_rate['bond_ticker']=='110032.SH']

,bond_ticker,start_date,end_date,rate
11,110032.SH,20160104,20170103,0.2000
12,110032.SH,20170104,20180103,0.5000
13,110032.SH,20180104,20190103,1.0000
14,110032.SH,20190104,20190319,1.5000


In [8]:
cbond_data['20170103']['110030.SH'].keys()

dict_keys(['date', 'open', 'high', 'low', 'close', 'volume', 'amount', 'pre_close', 'ovnt_ret', 'intra_ret', 'return', 'next_return', 'is_last', 'conv_price', 'rdmpt_bgndt', 'rdmpt_enddt', 'stock_ticker', 'adj_factor', 'top1500', 'adj_conv_price', 'adj_conv_share'])

In [9]:
stock_data['20170104']['600185.SH'].keys()

dict_keys(['BloombergID', 'date', 'adj_close', 'adj_open', 'adj_pre_close', 'adj_factor', 'vol100', 'vol250', 'value', 'mkt', 'top1500', 'CSI300', 'CSI800', 'limit_buy_open', 'limit_sell_open', 'limit_buy_close', 'limit_sell_close', 'is_unusual'])

#### 2. Run parameters

In [10]:
def timeDelta(start, end, nan=0):
    """
    helper function used to calculate time delta between start and end date
    """
    if pd.isna(start) or pd.isna(end):
        return nan
    start = pd.to_datetime(str(int(start)))
    end = pd.to_datetime(str(int(end)))
    return (end - start) / pd.Timedelta(days=365)


def getPara(date, bond_ticker, stock_ticker):
    """
    return convertible bond para info on given date
    """
    bond = cbond_data[str(date)][bond_ticker]
    stock = stock_data[str(date)][stock_ticker]
    info = cbond_info[cbond_info['bond_ticker']==bond_ticker].iloc[0].copy()
    rate = cbond_rate[(cbond_rate['end_date'] > date) & (cbond_rate['bond_ticker']==bond_ticker)].copy()

    para = {}
    para['date'] = str(date)
    para['bond_ticker'] = bond_ticker
    para['stock_ticker'] = stock_ticker
    
    para['maturity'] = timeDelta(date, rate['end_date'].values[-1])
    para['face_value'] = 100
    para['redemption'] = info['payment'] / 100
    para['coupon'] = dict(zip([timeDelta(date, d) for d in rate['end_date']], (rate['rate'] / 100).tolist()))

    para['conv_ratio'] = bond['adj_conv_share']
    para['conv_price'] = bond['adj_conv_price']
    para['conv_date'] = (0, para['maturity'])
    para['conv_adj'] = None
    
    para['call'] = info['call_price']
    para['call_date'] = (max(timeDelta(date, info['rdmpt_bgndt']), 0), timeDelta(date, info['rdmpt_enddt']))
    para['call_soft'] = (info['call_win'], info['call_min'], info['rdmpt_transit'] / 100)
    
    para['put'] = info['put_price']
    para['put_date'] = (max(timeDelta(date, info['rpchs_bgndt']), 0), timeDelta(date, info['rpchs_enddt']))
    para['put_soft'] = (info['put_win'], info['put_win'], info['rpchs_transit'] / 100)

    para['r'] = 0.08
    para['S'] = stock['adj_close']
    para['sigma'] = stock['vol100'] * np.sqrt(250)
    para['q'] = 0
    
    para['bondprice'] = bond['close']
    para['cr'] = 0.08
    return para

In [11]:
# create/load convertible bond paras
read = True   # read from existing pickle file if read=True, generate again if read==False
write = True  # whether to save para to pickle file

if read and os.path.exists(os.path.join(data_path, 'model_para.pkl')):
    paras = pickle.load(open(os.path.join(data_path, 'model_para.pkl'), 'rb'))

else:
    paras = []
    cbond_price = cbond_price.sort_values(by=['bond_ticker', 'date'])
    for i in trange(len(cbond_price)):
        if cbond_price['is_last'].iloc[i] == True:
            continue
        date = cbond_price['date'].iloc[i]
        bond_ticker = cbond_price['bond_ticker'].iloc[i]
        stock_ticker = cbond_price['stock_ticker'].iloc[i]
        para = getPara(date, bond_ticker, stock_ticker)
        paras.append(para)
    
    if write:
        pickle.dump(paras, open(os.path.join(data_path, 'model_para.pkl'), 'wb'))

#### 3. run hidden volatility

In [12]:
def fitSigma(para, init_overvalue):
    """
    return hidden volatility fitted by market price
    """
    overvalue = init_overvalue
    
    # update hidden volatility if absulate overvalue larger than 1%
    while (np.abs(overvalue) > 0.01) and (np.sign(init_overvalue) == np.sign(overvalue)):
        
        if np.abs(overvalue) <= 0.1: # set adjusting volatility parameter 
            beta = np.sqrt(np.abs(overvalue)) * np.sign(overvalue)
        else:
            beta = overvalue
        
        sigma = para['sigma']
        para['sigma'] = para['sigma'] * (1 + beta)
        price = MonteCarlo(para, trial=2000, step=100, cbond_type='AM', forward='milstein', backward='LSMC')
        
        if np.abs(para['bondprice'] / price - 1) >= np.abs(overvalue):  # break if failed to reduce the overvalue 
            return sigma
            break
    
        overvalue = para['bondprice'] / price - 1
    return para['sigma']

In [ ]:
# Load/Write hidden volatility at the beginning trade date of each ticker
read = True   # read from existing pickle file if read=True, generate again if read==False
write = True  # whether to save para to pickle file

if read and os.path.exists(os.path.join(data_path, 'model_hidden_vol.pkl')):
    hidden_vol = pickle.load(open(os.path.join(data_path, 'model_hidden_vol.pkl'), 'rb'))
    
else:
    bond_tickers = cbond_price.loc[cbond_price['is_last']==False, 'bond_ticker'].unique().tolist()

    hidden_vol = {}

    for bond_ticker in tqdm(bond_tickers):
        bond_data = cbond_price[cbond_price['bond_ticker']==bond_ticker].copy()
        para = getPara(bond_data['date'].iloc[0], bond_ticker, bond_data['stock_ticker'].iloc[0])
        price = MonteCarlo(para, trial=2000, step=100, cbond_type='AM', forward='milstein', backward='LSMC')
        overvalue = para['bondprice'] / price - 1
        hidden_beta = fitSigma(para, overvalue) / sigma
        hidden_vol[bond_ticker] = hidden_beta
    
    if write:
        pickle.dump(hidden_vol, open(os.path.join(data_path, 'model_hidden_vol.pkl'), 'wb'))

#### 4. Run pricing model

In [13]:
def runModel(data, num_range=(0, 5000)):
    """
    Version 1: run pricing model over each sample based on realized volatility
    """
    for para in tqdm(paras[num_range[0]: num_range[1]]):
        price = MonteCarlo(para, trial=2000, step=100, cbond_type='AM', forward='milstein', backward='LSMC')
        data['date'].append(int(para['date']))
        data['bond_ticker'].append(para['bond_ticker'])
        data['stock_ticker'].append(para['stock_ticker'])
        data['vol'].append(para['sigma'])
        data['close'].append(para['bondprice'])
        data['model'].append(price)

    data = pd.DataFrame(data)
    return data


def runModelV2(hidden_vol, data, bond_ticker):
    """
    Version 2: run pricing model over each sample using static hidden volatility adjustment
    """
    bond_paras = [para for para in paras if para['bond_ticker']==bond_ticker]
    
    for para in tqdm(bond_paras):
        para['sigma'] *= hidden_vol[para['bond_ticker']]
        price = MonteCarlo(para, trial=2000, step=100, cbond_type='AM', forward='milstein', backward='LSMC')
        
        data['date'].append(int(para['date']))
        data['bond_ticker'].append(para['bond_ticker'])
        data['stock_ticker'].append(para['stock_ticker'])
        data['vol'].append(para['sigma'])
        data['close'].append(para['bondprice'])
        data['model'].append(price)

    data = pd.DataFrame(data)
    return data


def runModelV3(hidden_vol, data, bond_ticker):
    """
    Version 3: run pricing model over each sample using dynamic hidden volatility adjustment
    """
    bond_paras = [para for para in paras if para['bond_ticker']==bond_ticker]
    
    for para in tqdm(bond_paras):
        para['sigma'] *= hidden_vol[para['bond_ticker']]
        sigma = para['sigma']
        price = MonteCarlo(para, trial=2000, step=100, cbond_type='AM', forward='milstein', backward='LSMC')
        overvalue = para['bondprice'] / price - 1
        hidden_beta = fitSigma(para, overvalue) / sigma
        hidden_vol[para['bond_ticker']] *= hidden_beta
        
        data['date'].append(int(para['date']))
        data['bond_ticker'].append(para['bond_ticker'])
        data['stock_ticker'].append(para['stock_ticker'])
        data['vol'].append(para['sigma'])
        data['close'].append(para['bondprice'])
        data['model'].append(price)

    data = pd.DataFrame(data)
    return data


def emptyData():
    """
    helper function, create an empty data container used in runModel
    """
    data = {'date': [],
       'bond_ticker':[],
       'stock_ticker':[],
       'vol':[],
       'close':[],
       'model':[],
       }
    return data

In [14]:
# run model
read = True  # read from existing csv file if read=True, generate again if read==False

model_path = os.path.join(data_path, 'model_V3')
if not os.path.exists(model_path):
    os.makedirs(model_path)

models = os.listdir(model_path)

cbond_data = cbond_price[cbond_price['is_last']==False].copy()
bond_tickers = cbond_data['bond_ticker'].unique().tolist()
paras = pickle.load(open(os.path.join(data_path, 'model_para.pkl'), 'rb'))
hidden_vol = pickle.load(open(os.path.join(data_path, 'model_hidden_vol.pkl'), 'rb'))

for i, bond_ticker in enumerate(bond_tickers):
    name = '{}.csv'.format(bond_ticker)
    if (read==False) or (name not in models):
        print('{} {}'.format(i, bond_ticker))
        model = runModelV3(hidden_vol, emptyData(), bond_ticker)
        model.to_csv(os.path.join(model_path, name), index=False)

#### 5. Read model

In [16]:
# read model
model_path = os.path.join(data_path, 'model_V3')
models = os.listdir(model_path)

model_data = []

for model in models:
    model_data.append(pd.read_csv(os.path.join(model_path, model), index_col=False, low_memory=False))
    
model_data = pd.concat(model_data)
model_data = model_data.sort_values(by=['bond_ticker', 'date']).reset_index(drop=True)

model_data['overvalue'] = model_data['close'] / model_data['model'] - 1
model_data.head()

,date,bond_ticker,stock_ticker,vol,close,model,overvalue
0,20170103,110030.SH,600185.SH,0.5442,114.0400,114.7751,-0.0064
1,20170104,110030.SH,600185.SH,0.5393,114.2300,110.1316,0.0372
2,20170105,110030.SH,600185.SH,0.5258,114.9500,113.9838,0.0085
3,20170106,110030.SH,600185.SH,0.5208,114.8600,114.1824,0.0059
4,20170109,110030.SH,600185.SH,0.5880,115.5300,112.4799,0.0271
